In [1]:
import csv

from pathlib import Path

from owlready2 import *

import pandas as pd

from onto_lib.onto_classes import *
from onto_lib.utils import create_csvs

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
# set up out path
data_p = Path("../../../data/")

# set paths
ontology_p = "../../../data/"

# set onto path
# upper_p   = ontology_p + "/nourish-merged-2.owl"
# upper_p   = ontology_p + "/usgovt-ontology.owl"
upper_p   = ontology_p + "/foodcomposite.owl"

In [3]:
# load ontologies
onto = get_ontology(upper_p).load()

* Owlready2 * WARNING: AnnotationProperty http://purl.obolibrary.org/obo/RO_0001900 belongs to more than one entity types: [owl.ObjectProperty, owl.AnnotationProperty, obo.RO_0002422]; I'm trying to fix it...
* Owlready2 * WARNING: AnnotationProperty http://purl.obolibrary.org/obo/RO_0040042 belongs to more than one entity types: [owl.ObjectProperty, owl.AnnotationProperty, obo.RO_0002422]; I'm trying to fix it...


In [4]:
annot_dict = initialize_annots(onto)

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/FOODON_00002511

* Owlready2 * Warning: ignoring cyclic subclass of/subproperty of, involving:
  http://purl.obolibrary.org/obo/FOODON_03315876



obo.GO_0016053
http://purl.obolibrary.org/obo/GO_0016053


TypeError: metaclass conflict: the metaclass of a derived class must be a (non-strict) subclass of the metaclasses of all its bases

In [5]:
path_dict = {"node": "./test_nodes.csv",
             "rel":  "./test_rels.csv",
            #  "pattern": out_p / "pattern.csv"
             }
create_csvs(path_dict, annot_list=list(annot_dict.keys()))

In [6]:
g = create_graph(onto, path_dict)

owl.Thing does not have equivalent_to or is_a properties.


In [7]:
class_node_df = pd.read_csv(path_dict["node"])
print(class_node_df.shape[0])
class_node_df.drop_duplicates().shape[0]

/var/folders/wd/y5szpxf50s70ykknr5mg_1n00000gn/T/ipykernel_28698/3036961189.py:1: DtypeWarning: Columns (4,5,7,8,9,10,11,12,13,14,15,16,17,18,19,20,23,24,25,26,27,28,29,30,31,34,36,37,38,39,40,41,42,43,44,45,49,52,54,55,57,58,59,60,61,62,63,64,65,67,71,72,73,74,75,77,78,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  class_node_df = pd.read_csv(path_dict["node"])


44666


44666

In [7]:
class_rel_df = pd.read_csv(path_dict["rel"])
print(class_rel_df.shape[0])
class_rel_df.drop_duplicates().shape[0]

59666


59666

In [20]:
class_rel_df = pd.read_csv(path_dict["rel"])
print(class_rel_df.shape[0])
class_rel_df.drop_duplicates().shape[0]

65411


59666

In [8]:
old_node_df = pd.read_csv(data_p / "out/nodes.csv")
print(old_node_df.shape[0])
old_node_df.drop_duplicates().shape[0]

50279


50279

In [8]:
old_rels_df = pd.read_csv(data_p / "out/rels.csv")
print(old_rels_df.shape[0])
old_rels_df.drop_duplicates().shape[0]

65314


65314

In [27]:
# b = 'BLANK_7515'
# if 'UPPER' not in b:
#     print(":)")
a = set([(1,1), (1,2), (1,1), (2,1)])
if (4,4) not in a:
    a.add((4,4))
a

{(1, 1), (1, 2), (2, 1), (4, 4)}

In [15]:
diff_set = set(old_node_df['node_id:ID'].tolist()).difference(class_node_df['node_id:ID'].tolist())
[a for a in diff_set if a if 'UPPER' not in a]

[]

In [17]:
type(g.root)

owlready2.prop.ObjectPropertyClass

In [2]:
def initialize_annots(onto):
    annot_set = set()
    for c in onto.classes():
        annot_set = annot_set | get_annots(c)

    annot_dict = {a: None for a in annot_set}
    return annot_dict

def get_annots(c):
    annot_set = set()
    for class_prop in c.get_class_properties():
        match class_prop:
            case owlready2.annotation.AnnotationPropertyClass():
                '''
                The examples why extracting the annotations need to be in a try block:
                ValueError: Cannot read literal of datatype 570!: obo.IAO_0000117, obo.IAO_0000119
                ValueError: Cannot read literal of datatype 571!: rdf-schema.label
                ValueError: Cannot read literal of datatype 576! 1.1.date
                ValueError: Cannot read literal of datatype 6103!: oboInOwl.hasDbXref
                '''

                try:
                    p_str = ""
                    for p_item in class_prop.__getitem__(c):
                        if p_item is None:
                            continue
                        if len(p_str) > 0:
                            p_str = p_str + "###" + str(p_item)
                        else:
                            p_str = str(p_item)
                    if p_str == "None":
                        continue
                    # check name of annotation
                    if class_prop.label.first() is None:
                        annot_str = str(class_prop)
                    else:
                        annot_str = class_prop.label.first()
                    annot_set.add(annot_str)
                except:
                    pass

            case owlready2.prop.ObjectPropertyClass() | owlready2.prop.DataPropertyClass():
                pass
            case _:
                raise TypeError(f"Unknown type: {type(class_prop)}")
    return annot_set

In [2]:
def old_get_annots(c):
    annot_set = set()
    for class_prop in c.get_class_properties():
        match class_prop:
            case owlready2.annotation.AnnotationPropertyClass():
                '''
                The examples why extracting the annotations need to be in a try block:
                ValueError: Cannot read literal of datatype 570!: obo.IAO_0000117, obo.IAO_0000119
                ValueError: Cannot read literal of datatype 571!: rdf-schema.label
                ValueError: Cannot read literal of datatype 576! 1.1.date
                ValueError: Cannot read literal of datatype 6103!: oboInOwl.hasDbXref
                '''

                try:
                    p_str = ""
                    for p_item in class_prop.__getitem__(c):
                        if p_item is None:
                            continue
                        if len(p_str) > 0:
                            p_str = p_str + "###" + str(p_item)
                        else:
                            p_str = str(p_item)
                    if p_str == "None":
                        continue
                    # check name of annotation
                    if class_prop.label.first() is None:
                        annot_str = str(class_prop)
                    else:
                        annot_str = class_prop.label.first()
                    annot_set.add(annot_str)
                except:
                    pass

            case owlready2.prop.ObjectPropertyClass() | owlready2.prop.DataPropertyClass():
                pass
            case _:
                raise TypeError(f"Unknown type: {type(class_prop)}")
    return annot_set

In [7]:
[annot_dict[k] for k in sorted(annot_dict.keys())]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [13]:
for c in onto.classes():
    # if c.name == "IAO_0000078": # curation status -> OneOf example
    if c.name == "FOODON_00002396": # synthetic vanilla extract
    # if c.name == "FOODON_00002403": #food_material
    # if c.name == "FOODON_00002114":
    # if c.name == "CDNO_0000001": # breaks cause no label
        break

In [14]:
print(f"label: {c.label}\nEquivalent to: {c.equivalent_to[0]}")

label: ['synthetic vanilla extract']
Equivalent to: Not(obo.RO_0001000.some(obo.FOODON_03411355)) & obo.BFO_0000051.some(obo.CHEBI_18346 | obo.CHEBI_48408)


In [15]:
# test making graph
entity_set = set()
transition_set = set()

# Add node and create graph
root_node = EntityNode(c, "Concept")
g = Graph(root_node, entity_set, transition_set)

# Add children to root node
if list(c.equivalent_to):
    for child in c.equivalent_to:
        parse_logic(child, root_node,
                    edge="equivalent_to", edge_direction="target",
                    restr_key=None, restr_value=None
                    )

elif list(c.is_a):
    for child in c.is_a:
        parse_logic(child, root_node,
                    edge="is_a", edge_direction="target",
                    restr_key=None, restr_value=None
                    )

else:
    print(f"{c} does not have equivalent_to or is_a properties.")

In [6]:
g.root.children

AttributeError: 'children' annotation property is not defined.

In [17]:
root_node.children[0]["node"].children

[{'node': <onto_lib.onto_classes.TransitionNode at 0x11b8aaaa0>,
  'direction': 'source',
  'rel_type': 'member_of',
  'rel_label': '',
  'restriction': None,
  'restriction_value': None},
 {'node': <onto_lib.onto_classes.TransitionNode at 0x11b8a9960>,
  'direction': 'source',
  'rel_type': 'member_of',
  'rel_label': '',
  'restriction': None,
  'restriction_value': None}]

In [19]:
def export_traverse(*, node_p, rel_p, node, node_set=set()):
    match node:
        case Graph():
            # initialize recursion
            # node_children = node.root.children
            node = node.root
            node_set = append_node_row(node_p, node, node_set)
        case EntityNode():
             # node_children = node.children
            pass
        case TransitionNode():
            # node_children = node.children
            # if transitionNode has no children -> return and do not write empty node
            if len(node.children) == 0:
                return
            
    for n in node.children:
        node_set = append_node_row(node_p, n["node"], node_set)
        append_edge_row(rel_p, node, n)
        export_traverse(node_p=node_p, rel_p=rel_p,node=n['node'], node_set=node_set)

def append_node_row(node_p, n, node_set):
    # node_id:ID,label:string[],iri,:LABEL
    # return [n.id, n.label, n.iri, n.neo_type]
    if n.id not in node_set:
        row = [n.id, n.label, n.iri, n.neo_type]
        node_set.add(n.id)

        with open(node_p, 'a') as f:
            w = csv.writer(f, delimiter=',')
            w.writerow(row)

    return node_set

def append_edge_row(rel_p, parent_n, child_n):
    # :START_ID,:END_ID,:TYPE,restriction,restriction_value,label
    if child_n["direction"] == "target":
        row = [parent_n.id, child_n["node"].id, child_n["rel_type"], 
                child_n['restriction'], child_n['restriction_value'],
                child_n['rel_label'],
               ]
    else:
        row = [child_n["node"].id, parent_n.id, child_n["rel_type"], 
                child_n['restriction'], child_n['restriction_value'],
                child_n['rel_label'],
               ]
        
    with open(rel_p, 'a') as f:
        w = csv.writer(f, delimiter=',')
        w.writerow(row)

In [20]:
path_dict

{'node': './nodes.csv', 'rel': './rels.csv'}

In [21]:
# export_traverse(g)
node_set = set()
export_traverse(node_p=path_dict["node"], rel_p=path_dict["rel"],node=g, node_set=node_set)

In [13]:
type(g)
g.root.children[0]

{'node': <onto_classes.TransitionNode at 0x1157d6ec0>,
 'direction': 'target',
 'rel_type': 'equivalent_to',
 'restriction': None,
 'restriction_value': None}

In [28]:
match g:
    case Graph():
        print(":)")
    case _:
        print(":(")

:)


In [ ]:
'''
Still need to add annotations

Now that graph is successfully created!
Write function to navigate through graph
    Need to check set of nodes before writing to node file!
        Can be harsh with transition nodes and throw an error if not unique?



    start at root and go through all children
    IF type is TRANSITIONNODE WITH NO CHILDREN
        prune
        remove node from transition list
    ELSE
        Check to see if ENTITYNODE is in set
        Write node row csv out
        Write rel row csv out
'''

In [ ]:
raise NotImplementedError

In [8]:
c.equivalent_to[0]

obo.ENVO_00010483 & obo.RO_0000087.some(obo.CHEBI_33290)

In [6]:
# create tree for food material
graph = EntityNode(c, "Concept")

# create AND node
and_trans_node = TransitionNode("and_0001", "AND")

# Add AND node to graph
graph.add_child(and_trans_node, direction="source", rel_type="equivalent_to")

# Add links to AND
envo_node = EntityNode(c.equivalent_to[0].is_a[0], "Concept") # obo.ENVO_00010483 -> No children
and_trans_node.add_child(envo_node, direction="source", rel_type="member_of")

# Add BLANK node
blank_trans_node = TransitionNode("BLANK_0001", "BLANK")

# Connect blank node to AND
and_trans_node.add_child(blank_trans_node, direction="target", rel_type="member_of")

# Create entity node and connect with blank node
restriction = c.equivalent_to[0].is_a[1]
chebi_node = EntityNode(restriction.value, "Concept") # obo.ENVO_00010483 -> No children
blank_trans_node.add_child(envo_node, direction="source", rel_type=restriction.property)

In [ ]:
'''
What we really want:
class Graph
    root: starting node
    entity: list
    transition: list
    
functions: export, prune, func to verify nodes not already in list
2 graphs:
    Main graph with complete node list
    Sub graph: each pattern
    
Other classes:
    set_children
        Entity -> check if equivalent_to or is_a is valid list
                  the launch point into creating graph
        Transition -> match on property type (AND, OR, NOT, etc)
                      Unwrap and add nodes + children as appropriate
        
    get_children: retrieve children of node
    
    add_node: Add checks for both entity and transition
        Transition -> Modify __init__ call to take one of those types and create node
    


'''
'''
trace: trace tree and name nodes
    No longer needed since transition nodes can take full name of their description

prune: verify all transition nodes have children
    Def needed to verify integrity of tree due to bad nodes
    
write: write tree out to node and rel csv files
    Still need to add annotations to EntityNodes
'''
# def trace(g):
    

In [12]:
graph.children[0]["node"].children

[{'node': <onto_classes.EntityNode at 0x11f64d4e0>,
  'direction': 'source',
  'rel_type': 'member_of'},
 {'node': <onto_classes.TransitionNode at 0x11f64d360>,
  'direction': 'target',
  'rel_type': 'member_of'}]

In [9]:
c.equivalent_to[0].is_a[1]

obo.RO_0000087.some(obo.CHEBI_33290)

In [10]:
type(c.equivalent_to[0].is_a[1])

owlready2.class_construct.Restriction

In [13]:
"BLANK_" + str(c.equivalent_to[0].is_a[1])

'BLANK_obo.RO_0000087.some(obo.CHEBI_33290)'

In [12]:
c.equivalent_to[0].is_a[1].property

obo.RO_0000087

In [14]:
c.equivalent_to[0].is_a[1].value

obo.CHEBI_33290

In [ ]:
# Add transition node


In [ ]:
# class EntityNode:
#     # def __init__(self, entity, neo_type, parent=None):
#     def __init__(self, entity, neo_type):
#         self.id = str(entity)
#         self.label = ';'.join(entity.label)
#         self.iri = entity.iri
#         self.neo_type = neo_type
#         self.children = []

In [ ]:
# def export_traverse(node):
#     match node:
#         case Graph():
#             # initialize recursion
#             node_children = node.root.children
#             print(node.root.id)
#             print('------------')
            
#             # write out edges and parent node
#             # TODO
#         case EntityNode() | TransitionNode():
#             node_children = node.children
            
#     for n in node_children:
#         print(n['node'].id)
#         print(create_node_row(n['node']))
#         print('vvvvvvvvvvvvv')
#         export_traverse(n['node'])
#         # write out edges
#         #TODO
        
#         # Go through node's children
#         # for child in n["node"].children:
#         #     print(f"\n{child['node'].id}")
#         #     print(create_node_row(child['node']))
#         #     print('+++++++++++++++')
#         #     export_traverse(child['node'])
            
# def create_node_row(n):
#     # node_id:ID,label:string[],iri,:LABEL
#     return [n.id, n.label, n.iri, n.neo_type]